In [41]:
import openai
import json

client = openai.OpenAI()
messages = []

In [42]:
def get_weather(city):
    return f"The weather in {city} is sunny."


FUNCTION_MAP = {
    "get_weather": get_weather,
}


In [43]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [44]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls and len(message.tool_calls) > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: '{function_name}' with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                print(f"Error parsing arguments: {arguments}")
                arguments = {}

            function_result = FUNCTION_MAP[function_name](**arguments)
            print(f"Function result: {function_result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_result,
                }
            )

        call_ai()

    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"Assistant: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano-2025-08-07",
        messages=messages,
        tools=TOOLS,
    )

    process_ai_response(response.choices[0].message)


In [45]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()


User: my name is jinwook
Assistant: Nice to meet you, Jinwook. How can I help you today? I can use your name in this chat if you’d like. If you need something specific, just tell me what it is (e.g., weather, planning, answering questions, translations).
User: what is the weather in Korea
Assistant: Weather in Korea varies by city, so I need a specific city to give you an accurate update.

Which city would you like the weather for? Seoul is a common choice, but I can fetch any city (e.g., Busan, Daegu, Incheon, etc.).

If you want, I can grab Seoul's current weather right now.
User: Daejeon
Calling function: 'get_weather' with arguments: {"city":"Daejeon"}
Function result: The weather in Daejeon is sunny.
Assistant: Currently sunny in Daejeon. Want the temperature and humidity, or a short forecast for the next few hours or days? I can also check another city if you’d like.


In [46]:
messages

[{'role': 'user', 'content': 'my name is jinwook'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Jinwook. How can I help you today? I can use your name in this chat if you’d like. If you need something specific, just tell me what it is (e.g., weather, planning, answering questions, translations).'},
 {'role': 'user', 'content': 'what is the weather in Korea'},
 {'role': 'assistant',
  'content': "Weather in Korea varies by city, so I need a specific city to give you an accurate update.\n\nWhich city would you like the weather for? Seoul is a common choice, but I can fetch any city (e.g., Busan, Daegu, Incheon, etc.).\n\nIf you want, I can grab Seoul's current weather right now."},
 {'role': 'user', 'content': 'Daejeon'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_8psApIIozqRYk6BHA29X4SS8',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Daejeon"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_8psApIIozqRYk6BHA